In [1]:
from datasets import Dataset
from sentence_transformers import SentenceTransformer, losses, InputExample, losses
from torch.utils.data import DataLoader
from datasets import Dataset
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import faiss
import pickle

In [2]:
df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")[:2800]
query_df = pd.read_excel("0_2799_질의응답데이터셋.xlsx")

In [3]:
# 청크 생성
documents = []
metadatas = []
for idx, row in df.iterrows():
    # chunk = "\n".join([f"[{col}] {row[col]}" for col in chunks_in])
    chunk = f"'{row['사무명']}'은 {row['사무개요']}"
    metadata = "\n".join([f"[{col}] {row[col]}" for col in df.columns])
    documents.append(chunk)
    metadatas.append(metadata)
print("청크:\n", documents[0])
print("\n메타데이터:\n", metadatas[0])

청크:
 '토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.

메타데이터:
 [분류번호] SG4CADM-2015
[사무명] 토지임야대장등본교부 다량신청
[사무개요] 토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.
[신청유형] 교부
[신청자 자격] 누구나신청가능
[수수료] nan
[수수료내역] nan
[소관부처] 국토교통부
[담당부서] 공간정보제도과
[접수방법] 인터넷
[접수/처리] nan
[근거법령] nan
[신청서] nan
[발급물 내용] nan
[구비서류] nan


In [4]:
# train, test split
test_df = query_df[query_df.index % 5 >= 3]
print(test_df.shape)
test_df.head()

(5600, 3)


,사무명,질문,답변
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...","토지임야대장등본을 다량으로 신청하시려면, 인터넷으로 신청이 가능합니다. 이 민원은 ..."
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...","여러 필지의 토지임야대장등본을 한 번에 신청하시려는군요! 이 민원은 ""토지임야대장등..."
8,공탁신청(공탁물지급청구 포함),4. 제가 대출금을 연체하여 채권자가 공탁을 요구하고 있습니다. 제가 공탁금을 지급...,"공탁금의 지급과 관련된 절차에 대해 설명드리겠습니다.\n\n먼저, 이 사무는 ""공탁..."
9,공탁신청(공탁물지급청구 포함),5. 공탁금을 지급받으려면 특정한 조건이나 절차가 있는지 알고 싶습니다. 만약 저에...,"공탁금을 지급받으시려면 다음과 같은 절차와 서류가 필요합니다.\n\n먼저, 이 사무..."
13,친양자입양취소신고,"4. 친양자 입양을 취소할 때, 법원에서 어떤 기준으로 결정을 내리는지 궁금합니다....","친양자 입양을 취소하는 과정에 대해 궁금하신 점을 설명드리겠습니다.\n\n먼저, 이..."


In [11]:
test_df['질문'][13]

'4. 친양자 입양을 취소할 때, 법원에서 어떤 기준으로 결정을 내리는지 궁금합니다. 이와 관련된 사례나 기준이 있으면 알려주실 수 있나요?'

In [ ]:
model = SentenceTransformer("MNRLoss_epoch30")
embeddings = model.encode(documents, show_progress_bar=True)

# FAISS 저장
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 인덱스와 메타데이터 저장
faiss.write_index(index, "epoch30_rag_index.faiss")
with open("epoch30_rag_metadata.pkl", "wb") as f:
    pickle.dump((documents, metadatas), f)

You try to use a model that was created with version 4.1.0, however, your version is 3.4.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





Batches:   0%|          | 0/88 [00:00<?, ?it/s]

In [6]:
# 검색함수 생성성
def search_similar_documents(query, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, top_k)
    return I

In [7]:
accuracy = 0
error = []
for idx, row in test_df.iterrows(): # query(14000행) 중 test_df 5600개개
    chunks_idx = search_similar_documents(row['질문']) # df에서 질문을 뽑은 행
    if idx//5  in chunks_idx:
        accuracy += 1
    else:
        error.append(idx)
print("Accuracy: ", accuracy/5600)

Accuracy:  0.9360714285714286


In [ ]:
for i in error[:5]:
    predict_idx = search_similar_documents(query_df['질문'][i])[0][0]
    print(f"질문:\n{query_df['질문'][i]}\n 실제 사무명:  {df['사무명'][i//5]}\n 아웃풋 사무명: {df['사무명'][predict_idx]}\n")

질문:
4. 천연기념물로 지정된 식물을 가정에서 기르고 싶은데, 이를 반입하기 위한 신고 절차가 어떻게 되는지 알 수 있을까요?
 실제 사무명:  천연기념물 동물·식물·광물의 반입 신고
 아웃풋 사무명: 천연기념물 국내 반입 신고

질문:
5. 약제비용 지급 신청을 했는데, 지급이 지연되고 있습니다. 이런 경우에는 어디에 문의해야 하며, 얼마나 기다려야 할까요?
 실제 사무명:  약제비용 지급 신청(위탁병원 감면대상자)
 아웃풋 사무명: 산재보험 진료비·약제비청구

질문:
5. 최근에 승강기 관련 교육을 수료했는데, 이 교육 이수를 경력으로 인정받으려면 어떻게 신고해야 하는지 알고 싶습니다.
 실제 사무명:  기술자 경력 변경 신고
 아웃풋 사무명: 기술자 경력신고

질문:
5. 경력 신고 후에 변경 사항이 생기면 어떻게 수정할 수 있나요? 예를 들어 직장 변경이나 자격증 취득 등에서 발생하는 변경 사항은 어떤 절차를 통해 신고해야 하나요?
 실제 사무명:  기술자 경력신고
 아웃풋 사무명: 건설기술인 경력(변경)신고

질문:
4. 과징금 분할 납부 신청서를 제출했는데, 제출 후 처리 기간이 얼마나 걸리는지 궁금합니다. 그리고 그 기간 동안 과징금의 납부는 어떻게 되는지도 알고 싶습니다.
 실제 사무명:  승강기 안전관리법령 위반에 따른 과징금 분할 납부 신청
 아웃풋 사무명: 생활화학제품 및 살생물제 과징금 분할납부 신청

